In [20]:
import salabim as sim

env = sim.Environment(time_unit='seconds')

#Shopping baskets and carts:
carts = sim.Resource('carts', capacity=45) #45
baskets = sim.Resource('baskets', capacity=500)#several hundred
cart_basket_distribution = sim.Pdf((carts, 0.8,baskets, 0.2))

#shopping categories:
# fruit_and_vegetables 
# meat_and_fish
# bread
# cheese_and_dairy
# canned_and_packed_food
# frozen_foods
# drinks
item_taking_distribution = sim.Uniform(20,30)

#Clerks
#Bread and cheese use resources as they are working via counters. Checkout likely has to use queue as the customers need to choose the smallest one
bread_clerks = sim.Resource('bread_clerks', capacity=4) #4 employees, 1-6 items takes 2 min
bread_time_distribution = sim.Exponential(2*60)
cheese_and_dairy_clerks = sim.Resource('bread_clerks', capacity=3) #3 employees, 1 min avg.
cheese_and_dairy_time_distribution = sim.Exponential(1*60)

checkouts = []
for i in range(3):
    checkouts.append(sim.Queue(f"checkout{i}")) #3, 1.1s per item avg. payment 40-60s
#Checkout item time
time_per_item_distribution = sim.Exponential(1.1)
payment_time_distribution = sim.Uniform(40, 60)

#Distributions of items per customer
fruit_and_vegetables_distribution = sim.Triangular(4, 22,10)  # min=4, mode=10, max=22
meat_and_fish_distribution = sim.Triangular(0, 9, 4)           # min=0, mode=4, max=9
bread_distribution = sim.Triangular(1, 10, 4)                  # min=1, mode=4, max=10
cheese_and_dairy_distribution = sim.Triangular(1, 11, 3)       # min=1, mode=3, max=11
canned_and_packed_food_distribution = sim.Triangular(6, 35, 17)# min=6, mode=17, max=35
frozen_foods_distribution = sim.Triangular(2, 19, 8)           # min=2, mode=8, max=19
drinks_distribution = sim.Triangular(1, 20, 9)                 # min=1, mode=9, max=20

#Route choice distribution

#Routes
route1 = [
"fruit_and_vegetables",
"meat_and_fish",
"bread",
"cheese_and_dairy",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #ABCDEF, 80% 

route2 = [
"meat_and_fish",
"bread",
"cheese_and_dairy",
"fruit_and_vegetables",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #BCDEFG, 20%

route_distribution = sim.Pdf((route1, 0.8, route2, 0.2))
class Clerk(sim.Component):
    def __init__(self, checkout, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.checkout = checkout
        
    def process(self):
        while True:
            while len(self.checkout) == 0:
                self.passivate()
            self.customer = self.checkout.pop()
            self.customer.activate()  # get the customer out of it's hold()
            self.hold(sum(time_per_item_distribution.sample() for _ in sum(self.customer.shopping_list.values()))+payment_time_distribution.sample()) #hold the customer for scanning all items and during payment
            self.customer.activate()  # signal the customer that's all's done

class Customer(sim.Component):
    def __init__(self, route, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.route = route_distribution.sample()
        self.shopping_list = {
            "fruit_and_vegetables": fruit_and_vegetables_distribution.sample(),
            "meat_and_fish": meat_and_fish_distribution.sample(),
            "bread": bread_distribution.sample(),
            "cheese_and_dairy": cheese_and_dairy_distribution.sample(),
            "canned_and_packed_food": canned_and_packed_food_distribution.sample(),
            "frozen_foods": frozen_foods_distribution.sample(),
            "drinks": drinks_distribution.sample(),
        }
        self.progress = 0

    def process(self):
        if self.progress == 0:
            self.start_shopping()
        elif self.progress == len(self.shopping_list):
            self.go_to_checkout()
        else:
            self.get_product(self.shopping_list.keys()[self.progress])
            
    def start_shopping(self):
        raise NotImplementedError
        
    def go_to_checkout(self):
        raise NotImplementedError
        #enter emptiest queue
        #let clerk take items and wait
        #return cart/basket
        
    def get_product(self, product):
        raise NotImplementedError
        #Move to product location #<- only needed in animation
        #if cheese or dairy, request clerk to fulfill.
        #Else: Retrieve as many products as needed
        #products_needed = self.shopping_list[product]
        #self.progress += 1

#customer generation
customer_distribution = [30, 80, 110, 90, 80, 70, 80, 90, 100, 120, 90, 40]
for index,i  in enumerate(customer_distribution):
    sim.ComponentGenerator(Customer, iat=sim.Exponential(i, time_unit='hours'), at=index*60*60, duration=60*60) #assumes time in seconds

#clerk generation
for i in checkouts:
    print(i)
    Clerk(checkout = i)
    

env.run(duration=60*60*12)
            

Queue (checkout0)
Queue (checkout1)
Queue (checkout2)


In [18]:
checkouts[0]

Queue (checkout0)

In [12]:
help(sim.Exponential)

Help on class Exponential in module salabim.salabim:

class Exponential(_Distribution)
 |  Exponential(mean: float = None, time_unit: str = None, rate: float = None, randomstream: Any = None, env: 'Environment' = None)
 |  
 |  exponential distribution
 |  
 |  Parameters
 |  ----------
 |  mean : float
 |      mean of the distribtion (beta)
 |  
 |      if omitted, the rate is used
 |  
 |      must be >0
 |  
 |  time_unit : str
 |      specifies the time unit
 |  
 |      must be one of "years", "weeks", "days", "hours", "minutes", "seconds", "milliseconds", "microseconds"
 |  
 |      default : no conversion
 |  
 |  
 |  rate : float
 |      rate of the distribution (lambda)
 |  
 |      if omitted, the mean is used
 |  
 |      must be >0
 |  
 |  randomstream: randomstream
 |      randomstream to be used
 |  
 |      if omitted, random will be used
 |  
 |      if used as random.Random(12299)
 |      it assigns a new stream with the specified seed
 |  
 |  env : Environment
 |  

In [8]:
help(sim.Triangular(4, 22,10))

Help on Triangular in module salabim.salabim object:

class Triangular(_Distribution)
 |  Triangular(low: float, high: float = None, mode: float = None, time_unit: str = None, randomstream: Any = None, env: 'Environment' = None)
 |  
 |  triangular distribution
 |  
 |  Parameters
 |  ----------
 |  low : float
 |      lowerbound of the distribution
 |  
 |  high : float
 |      upperbound of the distribution
 |  
 |      if omitted, low will be used, thus effectively a constant distribution
 |  
 |      high must be >= low
 |  
 |  mode : float
 |      mode of the distribution
 |  
 |      if omitted, the average of low and high will be used, thus a symmetric triangular distribution
 |  
 |      mode must be between low and high
 |  
 |  time_unit : str
 |      specifies the time unit
 |  
 |      must be one of "years", "weeks", "days", "hours", "minutes", "seconds", "milliseconds", "microseconds"
 |  
 |      default : no conversion
 |  
 |  
 |  randomstream: randomstream
 |      r

In [2]:
import salabim as sim
help(sim.Pdf)

Help on class Pdf in module salabim.salabim:

class Pdf(_Distribution)
 |  Pdf(spec: Union[Iterable, Dict], probabilities=None, time_unit: str = None, randomstream: Any = None, env: 'Environment' = None)
 |  
 |  Probability distribution function
 |  
 |  Parameters
 |  ----------
 |  spec : list, tuple or dict
 |      either
 |  
 |      -   if no probabilities specified:
 |  
 |          list/tuple with x-values and corresponding probability
 |          dict where the keys are re x-values and the values are probabilities
 |          (x0, p0, x1, p1, ...xn,pn)
 |  
 |      -   if probabilities is specified:
 |  
 |          list with x-values
 |  
 |  probabilities : iterable or float
 |      if omitted, spec contains the probabilities
 |  
 |      the iterable (p0, p1, ...pn) contains the probabilities of the corresponding
 |      x-values from spec.
 |  
 |      alternatively, if a float is given (e.g. 1), all x-values
 |      have equal probability. The value is not important.
 |  